In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm

In [6]:
df = pd.read_csv('../data/cleaned/cleaned_vessel_calls.csv')
df.drop(columns=['vessel.vesselName','Timestamp'])

,arrivalDate,sailingDate,place.placeName,place.unlocode,place.placeType,place.countryName,Event_Type,IMO,vessel.vesselType
0,2021-01-17T10:08:08,2021-01-20T03:44:00,Longkou Port,CNLKU,Port,People's Republic of China,ARRIVAL,9337391,crude oil tanker
1,2021-01-17T10:08:08,2021-01-20T03:44:00,Longkou Port,CNLKU,Port,People's Republic of China,DEPARTURE,9337391,crude oil tanker
2,2021-01-25T05:16:08,2021-01-27T02:04:05,Longkou Port,CNLKU,Port,People's Republic of China,ARRIVAL,9337391,crude oil tanker
3,2021-01-25T05:16:08,2021-01-27T02:04:05,Longkou Port,CNLKU,Port,People's Republic of China,DEPARTURE,9337391,crude oil tanker
4,2021-02-21T01:17:30,2021-02-22T23:00:19,Zhoushan Port,CNZOS,Port,People's Republic of China,ARRIVAL,9337391,crude oil tanker
...,...,...,...,...,...,...,...,...,...
344421,2023-08-25T14:44:08,2023-08-29T17:06:17,Leixoes,PTLEI,Port,Portugal,DEPARTURE,9336787,bulk carrier
344422,2023-09-14T22:22:15,2023-09-19T13:32:02,Gent (Ghent),BEGNE,Port,Belgium,ARRIVAL,9336787,bulk carrier
344423,2023-09-14T22:22:15,2023-09-19T13:32:02,Gent (Ghent),BEGNE,Port,Belgium,DEPARTURE,9336787,bulk carrier
344424,2023-09-22T11:25:42,2023-09-28T01:32:27,la Pallice,FRLPE,Sub Port,France,ARRIVAL,9336787,bulk carrier


In [3]:
df["arrivalDate"] = pd.to_datetime(df["arrivalDate"])
df["sailingDate"] = pd.to_datetime(df["sailingDate"])

In [5]:
df.sort_values(by=["IMO", "arrivalDate"], inplace=True)
df['next_unlocode'] = df.groupby('IMO')['place.unlocode'].shift(-1)
df['is_last_imo'] = df['IMO'] != df['IMO'].shift(-1)
df_cleaned = df[~df["is_last_imo"]]

,arrivalDate,sailingDate,place.placeName,place.unlocode,place.placeType,place.countryName,Timestamp,Event_Type,IMO,vessel.vesselName,vessel.vesselType,next_unlocode,is_last_imo
314760,2023-01-18 14:10:05,2023-01-26 04:05:00,Los Angeles,USLAX,Port,United States of America,2023-01-18T14:10:05,ARRIVAL,1026764,Max 08,Vessel Type (unspecified),USLAX,False
314761,2023-01-18 14:10:05,2023-01-26 04:05:00,Los Angeles,USLAX,Port,United States of America,2023-01-26T04:05:00,DEPARTURE,1026764,Max 08,Vessel Type (unspecified),CNJGY,False
314762,2023-02-22 07:17:58,2023-02-23 18:56:16,Jiangyin,CNJGY,Port,People's Republic of China,2023-02-22T07:17:58,ARRIVAL,1026764,Max 08,Vessel Type (unspecified),CNJGY,False
314763,2023-02-22 07:17:58,2023-02-23 18:56:16,Jiangyin,CNJGY,Port,People's Republic of China,2023-02-23T18:56:16,DEPARTURE,1026764,Max 08,Vessel Type (unspecified),CNTXG,False
314764,2023-03-02 16:00:39,2023-03-03 12:31:09,Tianjin Xingang Port,CNTXG,Port,People's Republic of China,2023-03-02T16:00:39,ARRIVAL,1026764,Max 08,Vessel Type (unspecified),CNTXG,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12574,2023-09-24 15:15:36,2023-09-25 08:23:38,Laem Chabang,THLCH,Port,Thailand,2023-09-24T15:15:36,ARRIVAL,9951135,Asterios,fully cellular containership,THLCH,False
12575,2023-09-24 15:15:36,2023-09-25 08:23:38,Laem Chabang,THLCH,Port,Thailand,2023-09-25T08:23:38,DEPARTURE,9951135,Asterios,fully cellular containership,SGSIN,False
12576,2023-09-28 11:12:12,2023-09-28 17:45:04,Singapore,SGSIN,Port,Republic of Singapore,2023-09-28T11:12:12,ARRIVAL,9951135,Asterios,fully cellular containership,SGSIN,False
12577,2023-09-28 11:12:12,2023-09-28 17:45:04,Singapore,SGSIN,Port,Republic of Singapore,2023-09-28T17:45:04,DEPARTURE,9951135,Asterios,fully cellular containership,MYPKG,False
